In [ ]:
import torch
from models.models_mae import mae_vit_huge_patch14_dec512d8b, MaskedAutoencoderViT

model_states = torch.load("../model_data/mae_model.bin")

#print(f"model_states: {model_states}")





In [ ]:
import collections

new_ordered_dict = []

for orig_key, val in model_states.items():
    key = orig_key.replace("_forward_module.model.", "")
    print(orig_key, key)
    new_ordered_dict.append((key, val))

new_ordered_dict = collections.OrderedDict(new_ordered_dict)
#new_ordered_dict

In [55]:

device = torch.device("cuda")
print("Device:", device, torch.cuda.is_available())
model = mae_vit_huge_patch14_dec512d8b()
print("--------------------------------------------------------------")
print(dir(model))
print("--------------------------------------------------------------")

model.load_state_dict(new_ordered_dict, strict=False)
model = model.to(device)


Device: cuda True
--------------------------------------------------------------
['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_init_weights', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict',

In [89]:
import torchvision.transforms as transforms
from transforms.mae_transforms import SquarePad
from PIL import Image

img_path = '../image_data/n01443537_12079.JPEG'
img_size = 224
#print(img_path)

tfs_train = transforms.Compose([
            SquarePad(),
            #transforms.Resize((args.img_size, args.img_size)),
            transforms.RandomResizedCrop(size=img_size, 
                                         scale=(0.7, 1.0), 
                                         ratio=(0.75, 1.3333333333333333)
                                         ),
            transforms.RandAugment(),
            transforms.RandomVerticalFlip(0.1),
            transforms.RandomHorizontalFlip(0.1),
            transforms.GaussianBlur(3),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ])


image = Image.open(img_path).convert("RGB")

image = tfs_train(image)

In [75]:
image.shape

torch.Size([1, 3, 224, 224])

In [90]:
import numpy as np
print(model.patch_embed.img_size)

print("1:", image.shape)

image = image.reshape(1, 3, 224, 224)

print("2:", image.shape)

image = image.to(device)

predictions = model(image)

(224, 224)
1: torch.Size([3, 224, 224])
2: torch.Size([1, 3, 224, 224])


In [91]:
predicted_image = predictions[1].cpu().detach().numpy()
predicted_image

array([[[-1.0324625 , -0.96839565, -1.0046719 , ..., -0.935914  ,
         -0.91667444, -0.9400685 ],
        [-1.0324469 , -0.96514314, -1.0056496 , ..., -0.94445115,
         -0.9094999 , -0.93577194],
        [-1.0236635 , -0.9530096 , -1.004213  , ..., -0.9399853 ,
         -0.8992717 , -0.94054145],
        ...,
        [-0.78094304, -0.73067164, -0.86981726, ..., -0.9161948 ,
         -0.9096669 , -0.99443096],
        [-0.87522614, -0.8051869 , -0.9219824 , ..., -0.91071576,
         -0.8990971 , -0.96212274],
        [-0.88441527, -0.81232077, -0.9280543 , ..., -0.8948136 ,
         -0.8880304 , -0.9575138 ]]], dtype=float32)

In [86]:
predicted_image[0].shape

(256, 588)

In [92]:
img = Image.fromarray(predicted_image[0], 'RGB')
img.save(img_path+".PNG")
print("Saved")
#img.show()

Saved


In [ ]:
import os
print(os.environ.get('CUDA_PATH'))

import torch

torch.cuda.is_available()
torch.cuda.device_count()